# Tratamento e geração do csv

In [106]:
!pip install googlemaps
!pip install odfpy

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import googlemaps
from datetime import datetime
from google.colab import userdata

In [ ]:
# # Código de tratamento
# for ano in range(2015, 2020):
#   if f'Acidentes {ano}.ods' not in os.listdir():
#     !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/Acidentes%20{ano}.ods

# for ano in range(2020, 2025):
#   if f'Acidentes {ano}.xls' not in os.listdir():
#     !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/Acidentes%20{ano}.xls

# dados = pd.DataFrame()

# for ano in range (2015, 2020):
#   dados = pd.concat([dados, pd.read_excel(f'Acidentes {ano}.ods', engine='odf')])

# dados1 = dados.copy()

# for ano in range(2020, 2025):
#   dados = pd.concat([dados, pd.read_excel(f'Acidentes {ano}.xls')])

# df = dados.sort_values(by=['DATA', 'HORA'])

# df['CRUZAMENTO'] = df['CRUZAMENTO'].astype(str) + df['CRUZAMENTO.1'].astype(str)
# df['LOGRADOURO'] = df['LOGRADOURO'].astype(str) + df['LOGRADOURO1'].astype(str)
# df = df.drop(columns=['LOGRADOURO1', 'CRUZAMENTO.1', 'Unnamed: 0', 2])
# df.rename(columns={'Nº': 'NUMERO'}, inplace=True)
# df.to_csv('acidentes.csv', index=False)

# def remove_nans(s):
#     s = str(s)
#     if s == 'nan':
#         return s
#     elif s.startswith("nan"):
#         return s[3:]  # Skip the first 3 characters if they are "nan"
#     elif s.endswith("nan"):
#         return s[:-3]  # Skip the last 3 characters if they are "nan"
#     else:
#       return s

# df['LOGRADOURO'] = df['LOGRADOURO'].apply(remove_nans)
# df['CRUZAMENTO'] = df['CRUZAMENTO'].apply(remove_nans)

# df['DATA'] = pd.to_datetime(df['DATA'])
# df['NUMERO'] = df['NUMERO'].astype(str)
# df['lat'] = ''
# df['lng'] = ''
# df['types'] = ''
# df['bairro'] = ''
# df.rename(columns={'DATA': 'data', 'HORA': 'hora', 'TEMPO': 'tempo',
#                    'TIPO_ACIDENTE': 'tipo_acidente', 'GRAVIDADE': 'gravidade',
#                    'LOGRADOURO': 'logradouro', 'NUMERO': 'numero', 'CRUZAMENTO': 'cruzamento'}, inplace=True)

In [ ]:
# gravidades = list(df['gravidade'].unique())

# df['gravidade'] = df['gravidade'].apply(lambda x: 'nan' if pd.isna(x) else x)
# df['gravidade'] = df['gravidade'].apply(lambda x: 'nan' if x == ' ' else x)

In [ ]:
# gravidades = list(df['gravidade'].unique())
# gravidades.remove('nan')
# df['temp'] = df['cruzamento']

# new_gravidade_values = []
# new_temp_values = []

# for index, row in df.iterrows():
#     temp_value = row['temp']
#     for gravidade in gravidades:
#         if str(gravidade) in temp_value:
#             new_gravidade_values.append(str(gravidade))
#             temp_value = temp_value.replace(str(gravidade), '')
#             break
#     else:
#         new_gravidade_values.append(np.nan)

#     new_temp_values.append(temp_value)

# df['temp'] = new_temp_values

# df['cruzamento'] = df['temp']
# df.drop(columns=['temp'], axis=1, inplace=True)

In [ ]:
# df[df['data'].dt.year == 2021]
# df['cruzamento'] = df['cruzamento'].apply(lambda x: 'nan' if x == '' else x)
# df['cruzamento'].unique()

In [ ]:
# !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/Acidentes%202021.xls

# df = pd.read_csv('acidentes.csv')
# df['data'] = pd.to_datetime(df['data'])
# df.sort_values(by=['data', 'hora'], inplace=True)
# df.reset_index(drop=True, inplace=True)

# df2021 = pd.read_excel('Acidentes 2021.xls')

# mask_2021 = df['data'].dt.year == 2021

# # Check if the number of rows matches
# if len(df2021) == sum(mask_2021):
#     # Update 'gravidade' column where the year is 2021
#     df.loc[mask_2021, 'gravidade'] = df2021['CRUZAMENTO'].values
# else:
#     print("Erro filtrando df")

In [ ]:
df = pd.read_csv('acidentes.csv')
df.sort_values(by=['data', 'hora'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [108]:
chave = userdata.get('chave')

gmaps = googlemaps.Client(key=chave)

In [113]:
for index, row in df.iterrows():
  if pd.isna(row['cruzamento']):
    local = str(row['numero']) + ' ' + str(row['logradouro'])
  else:
    local = str(row['logradouro']) + ' ' + str(row['cruzamento'])

  geocode_result = gmaps.geocode(f'{local}, Santos, SP, Brazil')

  if pd.isna(row['cruzamento']):
    if len(geocode_result[0]['address_components']) > 2 and 'long_name' in geocode_result[0]['address_components'][2]:
      df.at[index, 'bairro'] = geocode_result[0]['address_components'][2]['long_name']
    else:
      print(index)
      df.at[index, 'bairro'] = 'Bairro não encontrado'
  else:
    if len(geocode_result[0]['address_components']) > 2 and 'long_name' in geocode_result[0]['address_components'][2]:
      df.at[index, 'bairro'] = geocode_result[0]['address_components'][1]['long_name']
    else:
      print(index)
      df.at[index, 'bairro'] = 'Bairro não encontrado'

  df.at[index, 'types'] = geocode_result[0]['types']
  df.at[index, 'lat'] = geocode_result[0]['geometry']['location']['lat']
  df.at[index, 'lng'] = geocode_result[0]['geometry']['location']['lng']

df.head(10)

<ipython-input-113-f66d31116080>:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'São Jorge' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'bairro'] = geocode_result[0]['address_components'][1]['long_name']
<ipython-input-113-f66d31116080>:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'intersection' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'types'] = geocode_result[0]['types']


,data,hora,tempo,tipo_acidente,gravidade,logradouro,numero,cruzamento,lat,lng,types,bairro
0,2015-01-01,01:03:00,BOM,COLISÃO,C/ VÍTIMAS LEVES,Rua Domingos José Martins,0,Avenida Nossa Senhora de Fátima,-23.947971,-46.370689,intersection,São Jorge
1,2015-01-01,01:03:00,BOM,COLISÃO,C/ VÍTIMAS LEVES,Rua Aureliano Coutinho,0,Rua Castro Alves,-23.968639,-46.316946,[intersection],Embaré
2,2015-01-01,01:30:00,BOM,COLISÃO,S/ LESÃO,Avenida General Francisco Glycerio,0,Praça Nenê Ferreira Martins,-23.958715,-46.330590,[intersection],Gonzaga
3,2015-01-01,02:00:00,BOM,COLISÃO,S/ LESÃO,Rua Bolívia,0,Rua 1º de Maio,-23.976554,-46.302618,[intersection],Aparecida
4,2015-01-01,07:30:00,BOM,CHOQUE,C/ VÍTIMAS LEVES,Rua Torquato Dias,1947,NaN,-23.946493,-46.359032,[street_address],Morro Nova Cintra
5,2015-01-01,08:15:00,BOM,COLISÃO,S/ LESÃO,Avenida dos Bancários,0,Rua Enguaguaçú,-23.985284,-46.305335,[intersection],Ponta da Praia
6,2015-01-01,09:00:00,BOM,CHOQUE,S/ LESÃO,Avenida Doutor Epitácio Pessoa,114,NaN,-23.972427,-46.320878,[premise],Avenida Doutor Epitácio Pessoa
7,2015-01-01,10:00:00,BOM,CHOQUE,C/ VÍTIMAS LEVES,Avenida Conselheiro Nébias,0,Avenida Vicente de Carvalho,-23.972448,-46.324334,[intersection],Boqueirão
8,2015-01-01,10:35:00,BOM,COLISÃO,S/ LESÃO,Avenida Presidente Wilson,0,Avenida Senador Pinheiro Machado,-23.967792,-46.346001,[intersection],José Menino
9,2015-01-01,16:15:00,BOM,COLISÃO,S/ LESÃO,Avenida Bartholomeu de Gusmão,62,NaN,-23.976670,-46.316568,[premise],Aparecida


In [114]:
df.to_csv('acidenteslats.csv', index=False)

# Análise

In [31]:
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

if 'acidentes.csv' not in os.listdir():
  !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/acidentes.csv

dados = pd.read_csv('acidentes.csv')

dados.sort_values(by=['data', 'hora'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados['data'] = pd.to_datetime(dados['data'])
dados['dia_semana'] = (dados['data'].dt.dayofweek)
dias = {0: 2, 1: 3, 2: 4, 3: 5, 4: 6, 5: 7, 6: 1}
dados['dia_semana'] = dados['dia_semana'].map(dias)
dados.head(1)

,data,numero,lat,lng
count,33599,33599.000000,33599.000000,33599.000000
mean,2018-07-09 11:43:02.969731328,129.121164,-23.955460,-46.332588
min,2015-01-01 00:00:00,0.000000,-24.007469,-46.477227
25%,2016-03-31 00:00:00,0.000000,-23.967713,-46.343837
50%,2017-09-13 00:00:00,0.000000,-23.957266,-46.331659
75%,2020-08-12 00:00:00,107.000000,-23.943116,-46.320182
max,2024-05-31 00:00:00,65600.000000,-17.540031,-39.742181
std,NaN,605.696950,0.038283,0.040883


In [32]:
# df = dados.copy()
# m = folium.Map(location=[-23.959, -46.332], zoom_start=13)

# for index, row in df.iterrows():
#   folium.Marker(location=[row['lat'], row['lng']], popup=str(row['numero']) + ' ' + str(row['logradouro']) + ' ' + str(row['cruzamento'])).add_to(m)


In [ ]:
df = dados.copy()

dfano = df[df['data'].dt.year > 2017]
dfano = dfano[dfano['data'].dt.month == 1]
dfano = df

tl = folium.TileLayer('https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png', 
                      attr='<a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors; <a href="https://carto.com/attributions">CARTO</a>', #Make sure that the `attr` is not empty and contains valid HTML.
                      subdomains='abcd',
                      maxZoom=20,
                      control=False)

m = folium.Map(location=[-23.959, -46.332], zoom_start=13, tiles=tl)

heat_data = [[row['lat'], row['lng']] for index, row in dfano.iterrows()]
HeatMap(heat_data,radius=17, control=False).add_to(m)
m

In [ ]:
import pandas as pd
import plotly.express as px

# Data with latitude/longitude and values
df = dados.copy()

fig = px.density_mapbox(df, lat = 'lat', lon = 'lng',
                        radius = 5,
                        center = dict(lat = -23.959, lon = -46.332),
                        zoom = 13,
                        mapbox_style = 'open-street-map',
                        opacity = 0.7,
                        color_continuous_scale='rainbow')
fig.show()

In [42]:
# Heatmap por mês de cada ano

df = dados.copy()
df['mes'] = df['data'].dt.month

heat_data = []
for _, group in df.groupby(pd.Grouper(key='data', freq='M')):
    heat_data.append([[row['lat'], row['lng'], 1] for _, row in group.iterrows()])

index = list(range(len(heat_data)))

m = folium.Map(location=[-23.959, -46.332], zoom_start=13, height=500, width=1000)

hm = folium.plugins.HeatMapWithTime(heat_data, index=index, auto_play=True, max_opacity=0.3, radius=15, gradient={'0':'Navy', '0.25':'Blue','0.5':'Green', '0.75':'Yellow','1': 'Red'})
hm.add_to(m)

#m

In [ ]:
m = folium.Map(location=[-23.959, -46.332], zoom_start=13)

# Group data by date and create the required data format
heat_data = []
for date, group in df.groupby('data'):
    heat_data.append([[row['lat'], row['lng'], 1] for i, row in group.iterrows()])

# Get unique dates as index
index = df['data'].unique().tolist()

hm = folium.plugins.HeatMapWithTime(heat_data, index=index, auto_play=True, max_opacity=0.3)
hm.add_to(m)

m